In [1]:
from crawl_data import *
import pandas as pd
from datetime import date
import pytz
from ultils import camel_to_snake, unix_to_datetime_with_timezone
from data_warehouse import insert_dataframe_to_sql

encod_url


## 1. Dữ liệu danh sách các quỹ mở đang có


In [2]:
df_funds = get_all_funds()
df_funds.head(2)

,id,name,shortName,code,price,nav,lastYearNav,holdingVolume,firstIssueAt,approveAt,description,createAt,updateAt,owner,status,dataFundAssetType
0,68,QUỸ ĐẦU TƯ CỔ PHIẾU KINH TẾ HIỆN ĐẠI VINACAPITAL,VMEEF,VMPF,10000.0,14945.25,11250.56,8554587.13,1681146000000,1698976912410,Quỹ VMEEF được đầu tư chủ yếu vào các cổ phiếu...,1698976594493,1723625818886,"{'id': 677, 'encodeURL': 'cong-ty-co-phan-quan...",PRODUCT_ACTIVE,"{'id': 1, 'name': 'Quỹ cổ phiếu', 'code': 'STO..."
1,11,QUỸ ĐẦU TƯ LỢI THẾ CẠNH TRANH BỀN VỮNG SSI,SSISCA,SSISCA,10000.0,39648.93,30573.98,3064435.58,1411664400000,1566827410082,Quỹ tăng trưởng NAV thông qua việc chủ động và...,1566826497938,1723626286873,"{'id': 606, 'encodeURL': 'cong-ty-tnhh-quan-ly...",PRODUCT_ACTIVE,"{'id': 1, 'name': 'Quỹ cổ phiếu', 'code': 'STO..."


In [3]:
# get list of funds and list of owner
list_fund_id = list(df_funds.id)
list_owner = list(df_funds.owner.drop_duplicates())
print(
    "Number of funds: {}, number of owners: {}".format(
        len(list_fund_id), len(list_owner)
    )
)

Number of funds: 50, number of owners: 22


In [4]:
# convert from unix to datetime
list_datetime_col = ["firstIssueAt", "approveAt", "updateAt", "createAt"]
for col in list_datetime_col:
    df_funds[col] = df_funds[col].apply(lambda x: unix_to_datetime_with_timezone(x))


# add owner_id
df_funds["owner_id"] = df_funds["owner"].apply(lambda x: x["id"])

# add fund type and code
df_funds["fund_type_name"] = df_funds["dataFundAssetType"].apply(lambda x: x["name"])
df_funds["fund_type_code"] = df_funds["dataFundAssetType"].apply(lambda x: x["code"])

# rename column cammel format to snake
df_funds.columns = [camel_to_snake(col) for col in df_funds.columns]

# handle missing or NaN value
df_funds = df_funds.where(pd.notnull(df_funds), None)

df_funds = df_funds.drop(["data_fund_asset_type", "owner"], axis=1)

In [ ]:
# insert data to funds table
insert_dataframe_to_sql("funds", df_funds)

## 2. Danh sách các Tổ chức phát hành (owner)


In [12]:
df_owner = pd.DataFrame(list_owner)
df_owner.columns = [camel_to_snake(col) for col in df_owner.columns]

# handle missing or NaN value
df_owner = df_owner.where(pd.notnull(df_owner), None)

df_owner = df_owner[
    [
        "id",
        "encode_url",
        "code",
        "name",
        "email",
        "short_name",
        "address1",
        "website",
        "phone",
    ]
]

df_owner.head(2)

,id,encode_url,code,name,email,short_name,address1,website,phone
0,677,cong-ty-co-phan-quan-ly-quy-vinacapital_677,007F02838278535,CÔNG TY CỔ PHẦN QUẢN LÝ QUỸ VINACAPITAL,irwm@vinacapital.com,VINACAPITAL,"Lầu 17, 115 Nguyễn Huệ, Phường Bến Nghé, Quận ...",https://wm.vinacapital.com,02838278535
1,606,cong-ty-tnhh-quan-ly-quy-ssi_606,007F02838242897,CÔNG TY TNHH QUẢN LÝ QUỸ SSI,info@ssi.com.vn,SSIAM,"Tầng 5, 1C Ngô Quyền, Phường Lý Thái Tổ, Quận ...",https://www.ssi.com.vn,02838242897


In [13]:
# insert data to funds table
insert_dataframe_to_sql("owners", df_owner)

Connecting to SQL Server database....
connection successful.
Data inserted into owners successfully.


## 3. Danh sách top nắm giữ theo mã cổ phiếu/trái phiếu


In [14]:
top_holding = []
for fund_id in list_fund_id[:5]:
    top_holding.append(get_top_holding(fund_id))

# flatten list top hoding
top_holding = [stock_code for fund in top_holding for stock_code in fund]

In [16]:
df_top_holding = pd.DataFrame(top_holding)

df_top_holding["updateAt"] = df_top_holding["updateAt"].apply(
    lambda x: unix_to_datetime_with_timezone(x)
)

# rename column cammel format to snake
df_top_holding.columns = [camel_to_snake(col) for col in df_top_holding.columns]

df_top_holding = df_top_holding[
    [
        "id",
        "fund_id",
        "stock_code",
        "price",
        "change_from_previous",
        "change_from_previous_percent",
        "industry",
        "type",
        "net_asset_percent",
        "update_at",
    ]
]

df_top_holding.head()

,id,fund_id,stock_code,price,change_from_previous,change_from_previous_percent,industry,type,net_asset_percent,update_at
0,56777,68,FOX,93.2,1.90,2.08,Công nghệ và thông tin,STOCK,10.52,2024-08-08 06:00:16.489000+07:00
1,56791,68,VCB,91.7,0.20,0.22,Ngân hàng,STOCK,8.72,2024-08-08 06:00:16.489000+07:00
2,56771,68,ACB,24.8,0.15,0.61,Ngân hàng,STOCK,7.71,2024-08-08 06:00:16.489000+07:00
3,56778,68,FPT,134.8,1.10,0.82,Công nghệ và thông tin,STOCK,7.34,2024-08-08 06:00:16.489000+07:00
4,56772,68,BVH,45.5,0.15,0.33,Bảo hiểm,STOCK,4.56,2024-08-08 06:00:16.489000+07:00


In [17]:
# insert data to funds table
insert_dataframe_to_sql("top_stock_holdings", df_top_holding)

Connecting to SQL Server database....
connection successful.
Data inserted into top_stock_holdings successfully.


## 4. Danh sách top nắm giữ theo loại tài sản


In [21]:
top_asset = []
for fund_id in list_fund_id[:5]:
    top_asset.append(get_asset_hoding(fund_id))

# flatten list top hoding
top_asset = [asset_code for asset in top_asset for asset_code in asset]

In [24]:
df_top_asset = pd.DataFrame(top_asset)

df_top_asset["updateAt"] = df_top_asset["updateAt"].apply(
    lambda x: unix_to_datetime_with_timezone(x)
)

df_top_asset["createAt"] = df_top_asset["createAt"].apply(
    lambda x: unix_to_datetime_with_timezone(x)
)

# rename column cammel format to snake
df_top_asset.columns = [camel_to_snake(col) for col in df_top_asset.columns]

df_top_asset.head()

,id,asset_type,asset_percent,create_at,update_at,fund_id
0,1993,"{'id': 1, 'code': 'STOCK', 'name': 'Cổ phiếu',...",95.38,2024-08-08 06:00:16.519000+07:00,2024-08-08 06:00:16.519000+07:00,68
1,1994,"{'id': 3, 'code': 'CASH', 'name': 'Tiền và tươ...",4.62,2024-08-08 06:00:16.519000+07:00,2024-08-08 06:00:16.519000+07:00,68
2,2016,"{'id': 1, 'code': 'STOCK', 'name': 'Cổ phiếu',...",90.76,2024-08-08 06:00:25.101000+07:00,2024-08-08 06:00:25.101000+07:00,11
3,2017,"{'id': 3, 'code': 'CASH', 'name': 'Tiền và tươ...",9.24,2024-08-08 06:00:25.101000+07:00,2024-08-08 06:00:25.101000+07:00,11
4,2011,"{'id': 1, 'code': 'STOCK', 'name': 'Cổ phiếu',...",90.73,2024-08-08 06:00:23.885000+07:00,2024-08-08 06:00:23.885000+07:00,14


In [26]:
df_top_asset["asset_name"] = df_top_asset["asset_type"].apply(lambda x: x["name"])
df_top_asset["asset_type"] = df_top_asset["asset_type"].apply(lambda x: x["code"])

df_top_asset = df_top_asset[
    [
        "id",
        "fund_id",
        "asset_type",
        "asset_name",
        "asset_percent",
        "create_at",
        "update_at",
    ]
]

df_top_asset.head()

,id,fund_id,asset_type,asset_name,asset_percent,create_at,update_at
0,1993,68,STOCK,Cổ phiếu,95.38,2024-08-08 06:00:16.519000+07:00,2024-08-08 06:00:16.519000+07:00
1,1994,68,CASH,Tiền và tương đương tiền,4.62,2024-08-08 06:00:16.519000+07:00,2024-08-08 06:00:16.519000+07:00
2,2016,11,STOCK,Cổ phiếu,90.76,2024-08-08 06:00:25.101000+07:00,2024-08-08 06:00:25.101000+07:00
3,2017,11,CASH,Tiền và tương đương tiền,9.24,2024-08-08 06:00:25.101000+07:00,2024-08-08 06:00:25.101000+07:00
4,2011,14,STOCK,Cổ phiếu,90.73,2024-08-08 06:00:23.885000+07:00,2024-08-08 06:00:23.885000+07:00


In [30]:
# insert data to funds table
insert_dataframe_to_sql("top_asset_holdings", df_top_asset)

Connecting to SQL Server database....
connection successful.
Data inserted into top_asset_holdings successfully.


## 5. Danh sách top nắm giữ theo nhóm ngành


In [31]:
top_industry = []
for fund_id in list_fund_id[:5]:
    top_industry.append(get_industry_holding(fund_id))

# flatten list top hoding
top_industry = [industry for fund in top_industry for industry in fund]

In [32]:
df_top_industry = pd.DataFrame(top_industry)

# rename column cammel format to snake
df_top_industry.columns = [camel_to_snake(col) for col in df_top_industry.columns]

# create unique id value
df_top_industry["fund_index"] = df_top_industry.groupby("fund_id").cumcount() + 1

df_top_industry["id"] = df_top_industry.apply(
    lambda x: str(x["fund_id"]) + "_" + str(x["fund_index"]), axis=1
)

df_top_industry = df_top_industry[["id", "fund_id", "industry", "asset_percent"]]
df_top_industry.head(5)

,id,fund_id,industry,asset_percent
0,68_1,68,Ngân hàng,37.11
1,68_2,68,Công nghệ và thông tin,17.86
2,68_3,68,Xây dựng,6.33
3,68_4,68,Vận tải - Kho bãi,5.50
4,68_5,68,Bảo hiểm,4.56


In [33]:
# insert data to funds table
insert_dataframe_to_sql("top_industry_holdings", df_top_industry)

Connecting to SQL Server database....
connection successful.
Data inserted into top_industry_holdings successfully.


## 5. Net asset value


In [34]:
start_date = date.today().strftime("%Y%m%d")
print(start_date)

20240831


In [35]:
list_nav = []
for fund_id in list_fund_id[:5]:
    list_nav.append(get_nav_history(fund_id))

# flatten list top hoding
list_nav = [nav for fund in list_nav for nav in fund]

df_nav = pd.DataFrame(list_nav)

df_nav.head()

,id,createdAt,nav,navDate,productId
0,21001,NaN,9996.94,2023-05-09,68
1,20963,NaN,10000.00,2023-05-11,68
2,20964,NaN,10085.36,2023-05-31,68
3,20965,NaN,10041.77,2023-06-01,68
4,20966,NaN,10084.79,2023-06-02,68


In [36]:
df_nav.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3626 entries, 0 to 3625
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         3626 non-null   int64  
 1   createdAt  2344 non-null   float64
 2   nav        3626 non-null   float64
 3   navDate    3626 non-null   object 
 4   productId  3626 non-null   int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 141.8+ KB


In [37]:
df_nav["navDate"] = pd.to_datetime(df_nav["navDate"])
df_nav["createdAt"] = df_nav["createdAt"].apply(
    lambda x: None if pd.isna(x) else unix_to_datetime_with_timezone(x)
)

# rename column cammel format to snake
df_nav.columns = [camel_to_snake(col) for col in df_nav.columns]

df_nav.tail()

,id,created_at,nav,nav_date,product_id
3621,29046,2024-08-15 16:49:17.356000+07:00,12151.85,2024-08-15,46
3622,29149,2024-08-20 16:37:15.864000+07:00,12544.62,2024-08-20,46
3623,29221,2024-08-22 16:29:28.474000+07:00,12654.76,2024-08-22,46
3624,30007,2024-08-27 16:30:26.010000+07:00,12575.45,2024-08-27,46
3625,30094,2024-08-30 09:11:38.706000+07:00,12537.37,2024-08-29,46


In [ ]:
df_nav.info()